# **Practice 7**

1. In this practice we are going to program and use an explicit and an implicit method to solve an ODE.

+ Based on explicit Euler's method provided in the class' notebook program Heun's method.

+ Based on that same code and the Newton's method that you coded in practice 3, program the implicit Euler's method.

+ Solve the initial value 
$$y'=-\frac{3t^2y+y^2}{2t^3+3ty}$$
$$ y(1)=-2$$
on the interval $[1,2]$. Use step sizes $h = 0.0001, 0.001, 0.01, 0.1$. 

+ Use the implicit solution $t^3y^2+ty^3+4=0$ to check your results.

**Note:** Try to write functions that you could use for other problems with small changes, if any.

# Solution

### Imports

In [ ]:
import inspect
import sys
from typing import Callable

import numpy as np

### Newton's method

In [ ]:
def newton(err: float = 1e-4, f: 'Callable[float]' = None, f_dev: 'Callable[float]' = None, x0: float = 0) -> float:
    r"""Newton's method to find roots of a function.

    Args:
        err (float): Desired error of the method.
        f (Callable[float], optional): Analytical function to find its roots. Its input is the point to be evaluated in. Defaults to None.
        f_dev (Callable[float], optional): Analytical derivative of the function. Its input is the point to be evaluated in. Defaults to None.
        x0 (float, optional): Initial guess of the root.
            Note that an inadequate first guess could lead to undesired outputs such as no roots or undesired roots.
            Defaults to 0.
        
    Returns:
        float|None: Root of the function or None if the algorithm reaches its recursion limit.
    """
    iter, iter_dict = 0, {0: x0} # Not necessary to store all the values, but anyway
    limit = sys.getrecursionlimit()

    while True:
        if iter + 10 >= limit:
            print(f'Iteration limit ({limit}) reached without finding any root. Try with other initial guess or changing the recursion limit. \
                    Maybe there are no roots.')
            return

        iter_dict[iter+1] = iter_dict[iter] - f(iter_dict[iter]) / f_dev(iter_dict[iter])

        if abs(iter_dict[iter+1] - iter_dict[iter]) < err:
            return iter_dict[iter+1]

        iter += 1

## Exercise 1

In [ ]:
def euler_explicit(f: 'Callable[float, float]', y0: float, t0: float, t: float, h: float) -> np.ndarray:
    r"""Computes the explicit (forward) Euler method to solve ODEs.

    Args:
        f (Callable[float, float]): Function depending on y and t in that order.
            Equivalent to f(y,t).
        y0 (float): Initial value of the answer.
            Equivalent to y(t0).
        t0 (float): Initial time.
        t (float): Final time.
        h (float): Separation between the points of the interval.

    Returns:
        np.ndarray: Numerical solution of the ODE in the interval [t0, t0+h, ..., t-h, t].
    """
    t_ = np.arange(t0, t+h, h)
    N = len(t_)

    u = np.zeros_like(t_)
    u[0] = y0

    for i in range(N-1):
        u[i+1] = u[i] + h * f(u[i], t_[i])

    return u


def heun(f: 'Callable[float, float]', y0: float, t0: float, t: float, h: float) -> np.ndarray:
    """Computes Heun's method to solve ODEs.

    Args:
        f (Callable[float, float]): Function depending on y and t in that order.
            Equivalent to f(y,t).
        y0 (float): Initial value of the answer.
            Equivalent to y(t0).
        t0 (float): Initial time.
        t (float): Final time.
        h (float): Separation between the points of the interval.

    Returns:
        np.ndarray: Numerical solution of the ODE in the interval [t0, t0+h, t-h, t].
    """
    t_ = np.arange(t0, t+h, h)
    N = len(t_)

    u = np.zeros_like(t_)
    u[0] = y0

    for i in range(N-1):
        u[i+1] = u[i] + h/2 * (f(u[i]+h*f(u[i], t_[i]), t_[i+1]) + f(u[i], t_[i]))

    return u


def euler_implicit(f: 'Callable[float, float]', y0: float, t0: float, t: float, h: float, *args, **kwargs) -> np.ndarray:
    """Computes the implicit (backward) Euler method to solve ODEs.

    If `f` argument has an explicit dependence on y, Newton\' method is used to compute the next iteration the algorithm.
    Then, `err` must be passed as extra argument, and it is recommended to pass the analytical derivative of `f` as f_dev.
    In case this last `f_dev` is not passed, Newton\' method will use finite differences to numerically obtain it.

    See more about Newton\' method in module roots.

    Args:
        f (Callable[float, float]): Function depending on y and t in that order.
            Equivalent to f(y,t).
        y0 (float): Initial value of the answer.
            Equivalent to y(t0).
        t0 (float): Initial time.
        t (float): Final time.
        h (float): Separation between the points of the interval.

    Returns:
        np.ndarray: Numerical solution of the ODE in the interval [t0, t0+h, ..., t-h, t].
    """
    # ------ This part checks if the ode depends on y (to use newton) or only on t ----

    returns_of_f = inspect.getsource(f).split('return')
    lambdas_of_f = inspect.getsource(f).split('lambda')

    n_returns_of_f = len(returns_of_f)
    n_lambdas_of_f = len(lambdas_of_f)

    if n_returns_of_f > 2 or n_lambdas_of_f > 2:
        raise ValueError('Function `f` is not valid. It can only have one return or one lambda.')

    if n_returns_of_f < 2 and n_lambdas_of_f < 2:
        raise ValueError('Function `f` is not valid. It must have one return or one lambda.')

    elif n_returns_of_f == 2:
        function_of_f = returns_of_f[-1]

    elif n_lambdas_of_f == 2:
        function_of_f = lambdas_of_f[-1].split(':')[-1]

    # -----------

    t_ = np.arange(t0, t+h, h)
    N = len(t_)

    u = np.zeros_like(t_)
    u[0] = y0

    if 'y' in function_of_f:

        for i in range(N-1):
            def g(y): return u[i] - u[i+1] + h*f(y, t_[i+1])
            u[i+1] = newton(f=g, differentiator=forward, x0=u[i], *args, **kwargs)
    else:

        for i in range(N-1):
            u[i+1] = u[i] + h*f(y=None, t=t_[i+1])

    return u

In [1]:


if __name__ == '__main__':
    
    f_t_y = "- (3* t**2 * y + y**2) / (2* t**3 + 3* t*y)"
    h_vec = [0.0001, 0.001, 0.01, 0.1]

    for hi in h_vec:

        # print(f"Euler explicit wit h={hi} yields {euler_explicit(f_t_y, -2, 1, 2, hi)}")
        print(f"Euler implicit wit h={hi} yields {euler_implicit(f_t_y, -2, 1, 2, hi)}")    
        
    

ModuleNotFoundError: No module named 'practice03'